In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

import sys
sys.path.append("..")

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


import src.support.data_upload_support as dus
import time 

from unidecode import unidecode
import pandas as pd

Escrapear materiales y precios

In [11]:
keywords_data_competencia = pd.read_csv("../datos/keywords_data_competencia.csv", index_col=0)
keywords_data_competencia.head()

,keyword,urls_seo,name_seo,h3_seo,currency,avg__monthly_searches,cambio_en_tres_meses,cambio_interanual,competition,competition_(indexed_value),top_of_page_bid_(low_range),top_of_page_bid_(high_range),average_bid,rank_avg_monthly_searches,rank_cambio_en_tres_meses,rank_average_bid,score
0,tupper,NaN,NaN,NaN,EUR,18100.0,0.50,0.23,Alta,99.0,0.11,0.39,0.250,57,13,15,104.666667
1,tupper cristal,https://www.amazon.es/tupper-cristal/s?k=tuppe...,Amazon.es,Tupper Cristal,EUR,2900.0,0.83,0.22,Alta,100.0,0.14,0.37,0.255,56,23,13,107.333333
2,tupper cristal,https://www.lidl.es/es/tuppers/c6257?srsltid=A...,Compra Online | Lidl,"Tuppers para comida: de cristal, al vacío y más",EUR,2900.0,0.83,0.22,Alta,100.0,0.14,0.37,0.255,56,23,13,107.333333
3,tupper cristal,https://www.ikea.com/es/es/p/ikea-365-bote-con...,IKEA,"IKEA 365+ bote con tapa, rectangular vidrio ...",EUR,2900.0,0.83,0.22,Alta,100.0,0.14,0.37,0.255,56,23,13,107.333333
4,tupper de cristal,https://www.amazon.es/tupper-cristal/s?k=tuppe...,Amazon.es,Táper de plástico o de cristal: ¿cuál es mejor...,EUR,1000.0,0.82,0.23,Alta,100.0,0.13,0.31,0.220,54,21,24,118.000000


Escrapear

In [32]:
url_base_ohgar = "https://www.ohgar.com"
url_ohgar = "https://www.ohgar.com/es/c/recipientes-y-hermeticos--tapers/CP03005001"

driver = webdriver.Chrome()  # or specify path like: webdriver.Chrome(executable_path='/path/to/chromedriver')

driver.get(url_base_ohgar)

try:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(("css selector", "#onetrust-accept-btn-handler"))).click()
except:
    pass

urls_total = []
for pag in range(1,16):
    url_ohgar = f"https://www.ohgar.com/es/c/recipientes-y-hermeticos--tapers/CP03005001?page={pag}#product-list-anchor"
    driver.get(url_ohgar)
    time.sleep(2)
    page_source = driver.page_source
    sopa = BeautifulSoup(page_source, "html.parser")
    urls = [url_base_ohgar + elemento.find("a")["href"] for elemento in sopa.find_all("app-custom-product-grid-item")]
    urls_total.extend(urls)


Productos precios

In [ ]:
https://www.ohgar.com/es/p/recipiente-con-tapa-redondo-20cm-1l-vidrio-cookstore-ocuisine/4936310

In [ ]:

url_base_ohgar = "https://www.ohgar.com"

driver = webdriver.Chrome()

driver.get(url_base_ohgar)

try:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(("css selector", "#onetrust-accept-btn-handler"))).click()
except:
    pass

nombres_producto = []
precios = []
hermeticos = []
sostenibles = []
for url in urls:
    driver.get(url)
    driver.implicitly_wait(2)

    nombre_producto = re.findall(r"p/([^/]*)-\d+cm.", str(driver.current_url))
    precio = driver.find_element(By.TAG_NAME, "app-custom-product-summary")
    hermetico = 1 if "hermetico" in unidecode(driver.find_element(By.TAG_NAME, "app-custom-tab-paragraph-container").text.lower()) else 0
    sostenible = 1 if any(["ecologico" in unidecode(driver.find_element(By.TAG_NAME, "app-custom-tab-paragraph-container").text.lower()),
            "sostenible" in unidecode(driver.find_element(By.TAG_NAME, "app-custom-tab-paragraph-container").text.lower())]) else 0



In [ ]:
# Define number of drivers based on threads
MAX_DRIVERS = 5  # Adjust based on system capabilities and URLs

# Define lambda functions for each field with try-except to insert np.nan on failure
scraping_lambdas = {
    "url": lambda driver: driver.current_url,
    "precio": lambda driver: driver.find_element(By.TAG_NAME, "app-custom-product-summary").text.strip(),
    "hermetico": lambda driver: 1 if "hermetico" in unidecode(driver.find_element(By.TAG_NAME, "app-custom-tab-paragraph-container").text.lower()) else 0,
    "sostenible": lambda driver: 1 if any(keyword in unidecode(driver.find_element(By.TAG_NAME, "app-custom-tab-paragraph-container").text.lower()) for keyword in ["ecologico", "sostenible"]) else 0
}

# Initialize dictionary to store results as lists
total_dictionary = {key: [] for key in scraping_lambdas.keys()}

# Initialize WebDriver instances and accept cookies
drivers = []
def initialize_driver():
    driver = webdriver.Chrome()
    driver.get(url_base_ohgar)
    try:
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "#onetrust-accept-btn-handler"))
        ).click()
    except:
        pass  # Ignore if the cookie button is not found or already accepted
    return driver

# Use ThreadPoolExecutor to initialize drivers concurrently
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_DRIVERS) as executor:
    # Submit tasks to initialize drivers
    futures = [executor.submit(initialize_driver) for _ in range(MAX_DRIVERS)]
    
    # Collect initialized drivers
    for future in concurrent.futures.as_completed(futures):
        drivers.append(future.result())

# Function to scrape data from each URL using a specific driver
def scrape_with_driver(driver, url):
    driver.get(url)
    driver.implicitly_wait(2)  # Set implicit wait for each driver

    # Populate data for each field using lambda functions
    result = {}
    for key, func in scraping_lambdas.items():
        try:
            result[key] = func(driver)
        except:
            result[key] = np.nan  # Insert np.nan if any error occurs
    return result

# Use ThreadPoolExecutor to scrape concurrently with available drivers
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_DRIVERS) as executor:
    futures = []
    for i, url in enumerate(urls_total):
        print(url)
        driver = drivers[i % MAX_DRIVERS]  # Reuse available drivers in a round-robin fashion
        futures.append(executor.submit(scrape_with_driver, driver, url))

    # Collect results as each thread completes
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        for key, value in result.items():
            total_dictionary[key].append(value)

# Close all WebDriver instances
for driver in drivers:
    driver.quit()

# Print or process the total dictionary
print(total_dictionary)


https://www.ohgar.com/es/p/juego-2-recipientes-hermeticos-rectangular-38-y-122cl-pure-box-active-luminarc/4189186
https://www.ohgar.com/es/p/recipiente-hermetico-cuadrado-76cl-rojo-pure-box-luminarc/4474708
https://www.ohgar.com/es/p/recipiente-hermetico-rectangular-122cl-rojo-pure-box-luminarc/4474711
https://www.ohgar.com/es/p/recipiente-con-tapa-redondo-20cm-1l-vidrio-cookstore-ocuisine/4936310
https://www.ohgar.com/es/p/recipiente-vidrio-redondo-con-tapa-26cm-23l-cookstore-ocuisine/4936311
https://www.ohgar.com/es/p/recipiente-cuadrado-con-tapa-vidrio-20x17x55cm-1l-cookstore-ocuisine/4936313
https://www.ohgar.com/es/p/recipiente-rectangular-con-tapa-23x15x65cm-11l-vidrio-cookstore-ocuisine/4936316
https://www.ohgar.com/es/p/recipiente-rectangular-vidrio-40x27cm-45lcon-tapa-cookstore-ocuisine/4937757
https://www.ohgar.com/es/p/recipiente-hermetico-cuadrado-122cl-rojo-pure-box-luminarc/4474709
https://www.ohgar.com/es/p/recipiente-hermetico-rectangular-82cl-rojo-pure-box-luminarc/447

In [147]:
df_guardado = pd.DataFrame(total_dictionary)

In [148]:
df_guardado

,url,precio,hermetico,sostenible
0,https://www.ohgar.com/es/p/recipiente-hermetic...,"8,25 €",0.0,0.0
1,https://www.ohgar.com/es/p/recipiente-hermetic...,"7,00 €",0.0,0.0
2,https://www.ohgar.com/es/p/recipiente-vidrio-r...,"10,90 €",0.0,0.0
3,https://www.ohgar.com/es/p/recipiente-rectangu...,"16,00 €",0.0,0.0
4,https://www.ohgar.com/es/p/recipiente-rectangu...,"7,75 €",0.0,0.0
...,...,...,...,...
169,https://www.ohgar.com/es/p/recipiente-hermetic...,"8,50 €",1.0,0.0
170,https://www.ohgar.com/es/p/set-4-recipientes-3...,"11,00 €",0.0,0.0
171,https://www.ohgar.com/es/p/set-3-recipientes-4...,"7,00 €",0.0,0.0
172,https://www.ohgar.com/es/p/recipiente-hermetic...,"8,90 €",1.0,0.0


In [157]:
df_guardado["precio"] = pd.to_numeric(df_guardado["precio"].str.replace(" €","").str.replace(",","."),errors="coerce")

In [160]:
df_guardado.to_csv("../datos/competidores/ohgar_tuppers.csv")

In [161]:
dus.upload_to_drive("../datos/competidores/ohgar_tuppers.csv", '1vfttgGDHa3-eCMEN79QAue6aI3Gc1vvp')

Archivo subido con ID: 1BxVmVkQ6_OZw9ggL_SQGlRbHM72Q_kFT


Sacar nombres de producto